# Overview

In this notebook, I conduct the design and development of artificial intelligence models. The main goal of it is to try to predict the physical exercise adherence of the users based on how much time they have been exercising. 

**Author**: Jon Maestre Escobar

**Email**: jonmaestre@opendeusto.es.

In [2]:
import pandas as pd
import numpy as np
import warnings
import pandas as pd
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import time
from utilities import Data_cleaning

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.express.colors import sample_colorscale

import math
import copy
import re
%matplotlib inline

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
df_filtered_merged = pd.read_hdf('../data/filtered_merged_dataset_v1.h5', key='df')
df_filtered_merged.fillna(0, inplace=True)
df_filtered_merged.shape

(46020, 8705)

## **Users Filtered by Different Training Periods**

In this section, I have undertaken a detailed process to filter and analyze users based on the duration of their training periods recorded in our dataset. My objective was to categorize users into different groups based on their training consistency and ensure comprehensive data analysis by focusing on the range of days from the first to the last recorded training session.

**Steps Taken**

- **Initial Filtering**:
    - **30 Days or Less**: I first identified users whose training period spans 30 days or less from their first recorded training session to their last.
    - **31 to 90 Days**: Next, I filtered users whose training period spans more than 30 days but not exceeding 90 days.
    - **91 to 180 Days**: Similarly, I filtered users whose training period spans more than 90 days but up to 180 days.
    - **181 to 365 Days**: Finally, I identified users whose training period spans more than 180 days but not exceeding 365 days.

- **Data Completion**:
    - For each of these groups, I ensured that the training records were complete. Specifically, I filled in any missing days with zeros from the user's first recorded training day to ensure there are no gaps in the data.
    - This involved generating a complete date range for each user based on their first training session and merging it with the existing records, filling in the missing entries.

- **Detailed Analysis**:
    - The filtered and completed datasets were then analyzed to understand user behavior and training patterns better. This step helps in identifying trends and making data-driven recommendations for improving user engagement and training adherence.

By implementing these steps, I aim to provide a robust analysis framework that accurately reflects users' training habits, allowing us to draw meaningful insights and support the development of personalized fitness interventions. This comprehensive approach ensures that our analysis accounts for all training days, even those that were not initially recorded, providing a more accurate picture of user activity.

In [3]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is 30 days or less
users_30_days_or_less = user_training_period[user_training_period['training_period_days'] <= 30]

# Create a DataFrame with the filtered users
df_filtered_30days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_30_days_or_less['user_programs_user_id'])]
df_filtered_30days.shape

(4596, 8705)

In [4]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is between 30 and 90 days
users_30_to_90_days = user_training_period[(user_training_period['training_period_days'] > 30) & (user_training_period['training_period_days'] <= 90)]

# Create a DataFrame with the filtered users
df_filtered_30_to_90_days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_30_to_90_days['user_programs_user_id'])]
df_filtered_30_to_90_days.shape

(7006, 8705)

In [5]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is between 90 and 180 days
users_90_to_180_days = user_training_period[(user_training_period['training_period_days'] > 90) & (user_training_period['training_period_days'] <= 180)]

# Create a DataFrame with the filtered users
df_filtered_90_to_180_days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_90_to_180_days['user_programs_user_id'])]
df_filtered_90_to_180_days.shape

(13243, 8705)

In [6]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is between 180 and 365 days
users_180_to_365_days = user_training_period[(user_training_period['training_period_days'] > 180) & (user_training_period['training_period_days'] <= 365)]

# Create a DataFrame with the filtered users
df_filtered_180_to_365_days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_180_to_365_days['user_programs_user_id'])]
df_filtered_180_to_365_days.shape

(21175, 8705)

In [7]:
print(f'The number of user who have trained 30 days or less is:', df_filtered_30days.user_programs_user_id.nunique()) 
print(f'The number of user who have trained between 31 and 90 days is:', df_filtered_30_to_90_days.user_programs_user_id.nunique())
print(f'The number of user who have trained between 91 and 180 days is', df_filtered_90_to_180_days.user_programs_user_id.nunique())
print(f'The number of user who have trained between 181 and 365 days is', df_filtered_180_to_365_days.user_programs_user_id.nunique())

The number of user who have trained 30 days or less is: 1658
The number of user who have trained between 31 and 90 days is: 549
The number of user who have trained between 91 and 180 days is 572
The number of user who have trained between 181 and 365 days is 409


### **Complete With Zeros Non-Training Days**

In this part of the analysis, I focused on ensuring that each user's training record is complete by filling in non-training days with zeros. This step is crucial for accurate time series analysis and to avoid any misinterpretation of the user's training consistency. Here are the detailed steps I followed:

- **Identify First and Last Training Day**:
    - For each user in the filtered datasets, I identified the first and last day they recorded a training session. This serves as the basis for generating a complete date range for each user.

- **Generate Complete Date Ranges**:
    - Using the first training day as the starting point, I generated a complete date range for each user up to the specified number of days (30 days for users with up to 30 days of training, 90 days for users with training periods of 31-90 days, 180 days for users with training periods of 91-180 days, and 365 days for users with training periods of 181-365 days).

- **Merge with Original Data and Fill Missing Days**:
    - I merged these generated date ranges with the original user training data to identify days that were not recorded.
    - For the missing days, I filled in the relevant columns with zeros, ensuring that there are no gaps in the training data.

- **Combine Completed Data**:
    - The completed data for each user was then combined into a single DataFrame. This new DataFrame includes all the original training data as well as the newly added rows for the non-training days filled with zeros.

This process ensures that each user has a continuous record of training activity, allowing for more accurate analysis and modeling. By filling in the non-training days with zeros, I can better understand user behavior and training patterns, which is essential for developing effective AI models and making informed recommendations.

In [12]:
# Ensure that 'date' is in datetime format
df_filtered_30days['date'] = pd.to_datetime(df_filtered_30days['date'])

# Get the first training day for each user
first_training_day = df_filtered_30days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_30days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day['user_programs_user_id'], first_training_day['first_training_date']):
    # Generate a date range from the first day up to 30 days later
    date_range = pd.date_range(start=first_day, periods=30)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_30days[df_filtered_30days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_30days = pd.concat([completed_data_30days, user_data])

# Reset the index of the final DataFrame
completed_data_30days.reset_index(drop=True, inplace=True)
completed_data_30days.head()

,user_programs_user_id,date,session_executions_updated_at,1 leg bridge (left)_reps_1,1 leg bridge (left)_reps_10,1 leg bridge (left)_reps_11,1 leg bridge (left)_reps_12,1 leg bridge (left)_reps_13,1 leg bridge (left)_reps_14,1 leg bridge (left)_reps_15,1 leg bridge (left)_reps_16,1 leg bridge (left)_reps_17,1 leg bridge (left)_reps_2,1 leg bridge (left)_reps_3,1 leg bridge (left)_reps_4,1 leg bridge (left)_reps_5,1 leg bridge (left)_reps_6,1 leg bridge (left)_reps_7,1 leg bridge (left)_reps_8,1 leg bridge (left)_reps_9,1 leg bridge (left)_time_1,1 leg bridge (left)_time_10,1 leg bridge (left)_time_11,1 leg bridge (left)_time_12,1 leg bridge (left)_time_13,1 leg bridge (left)_time_14,1 leg bridge (left)_time_15,1 leg bridge (left)_time_16,1 leg bridge (left)_time_17,1 leg bridge (left)_time_2,1 leg bridge (left)_time_3,1 leg bridge (left)_time_4,1 leg bridge (left)_time_5,1 leg bridge (left)_time_6,1 leg bridge (left)_time_7,1 leg bridge (left)_time_8,1 leg bridge (left)_time_9,1 leg bridge (right)_reps_1,1 leg bridge (right)_reps_10,1 leg bridge (right)_reps_11,1 leg bridge (right)_reps_12,1 leg bridge (right)_reps_13,1 leg bridge (right)_reps_14,1 leg bridge (right)_reps_15,1 leg bridge (right)_reps_16,1 leg bridge (right)_reps_17,1 leg bridge (right)_reps_2,1 leg bridge (right)_reps_3,1 leg bridge (right)_reps_4,1 leg bridge (right)_reps_5,1 leg bridge (right)_reps_6,1 leg bridge (right)_reps_7,1 leg bridge (right)_reps_8,1 leg bridge (right)_reps_9,1 leg bridge (right)_time_1,1 leg bridge (right)_time_10,1 leg bridge (right)_time_11,1 leg bridge (right)_time_12,1 leg bridge (right)_time_13,1 leg bridge (right)_time_14,1 leg bridge (right)_time_15,1 leg bridge (right)_time_16,1 leg bridge (right)_time_17,1 leg bridge (right)_time_2,1 leg bridge (right)_time_3,1 leg bridge (right)_time_4,1 leg bridge (right)_time_5,1 leg bridge (right)_time_6,1 leg bridge (right)_time_7,1 leg bridge (right)_time_8,1 leg bridge (right)_time_9,1 leg dead lift (left)_reps_1,1 leg dead lift (left)_reps_10,1 leg dead lift (left)_reps_11,1 leg dead lift (left)_reps_12,1 leg dead lift (left)_reps_13,1 leg dead lift (left)_reps_14,1 leg dead lift (left)_reps_15,1 leg dead lift (left)_reps_2,1 leg dead lift (left)_reps_3,1 leg dead lift (left)_reps_4,1 leg dead lift (left)_reps_5,1 leg dead lift (left)_reps_6,1 leg dead lift (left)_reps_7,1 leg dead lift (left)_reps_8,1 leg dead lift (left)_reps_9,1 leg dead lift (left)_time_1,1 leg dead lift (left)_time_10,1 leg dead lift (left)_time_11,1 leg dead lift (left)_time_12,1 leg dead lift (left)_time_13,1 leg dead lift (left)_time_14,1 leg dead lift (left)_time_15,1 leg dead lift (left)_time_2,1 leg dead lift (left)_time_3,1 leg dead lift (left)_time_4,1 leg dead lift (left)_time_5,1 leg dead lift (left)_time_6,1 leg dead lift (left)_time_7,1 leg dead lift (left)_time_8,1 leg dead lift (left)_time_9,1 leg dead lift (right)_reps_1,1 leg dead lift (right)_reps_10,1 leg dead lift (right)_reps_11,1 leg dead lift (right)_reps_12,1 leg dead lift (right)_reps_13,1 leg dead lift (right)_reps_14,1 leg dead lift (right)_reps_15,1 leg dead lift (right)_reps_2,1 leg dead lift (right)_reps_3,1 leg dead lift (right)_reps_4,1 leg dead lift (right)_reps_5,1 leg dead lift (right)_reps_6,1 leg dead lift (right)_reps_7,1 leg dead lift (right)_reps_8,1 leg dead lift (right)_reps_9,1 leg dead lift (right)_time_1,1 leg dead lift (right)_time_10,1 leg dead lift (right)_time_11,1 leg dead lift (right)_time_12,1 leg dead lift (right)_time_13,1 leg dead lift (right)_time_14,1 leg dead lift (right)_time_15,1 leg dead lift (right)_time_2,1 leg dead lift (right)_time_3,1 leg dead lift (right)_time_4,1 leg dead lift (right)_time_5,1 leg dead lift (right)_time_6,1 leg dead lift (right)_time_7,1 leg dead lift (right)_time_8,1 leg dead lift (right)_time_9,1 leg dead lift 1with KB (right)_reps_1,1 leg dead lift 1with KB (right)_reps_10,1 leg dead lift 1with KB (right)_reps_11,1 leg dead lift 1with KB (right)_reps_12,1 leg dead lift 1with K

In [13]:
completed_data_30days.to_hdf('../data/completed_data_30days_v1.h5', key='df')

In [3]:
completed_data_30days = pd.read_hdf('../data/completed_data_30days_v1.h5', key='df')
print(f'30 days dataset shape:', completed_data_30days.shape)

30 days dataset shape: (49740, 8705)


In [15]:
# Ensure that 'date' is in datetime format
df_filtered_30_to_90_days['date'] = pd.to_datetime(df_filtered_30_to_90_days['date'])

# Get the first training day for each user
first_training_day_30_to_90 = df_filtered_30_to_90_days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day_30_to_90.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_30_to_90_days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day_30_to_90['user_programs_user_id'], first_training_day_30_to_90['first_training_date']):
    # Generate a date range from the first day up to 90 days later
    date_range = pd.date_range(start=first_day, periods=90)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_30_to_90_days[df_filtered_30_to_90_days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_30_to_90_days = pd.concat([completed_data_30_to_90_days, user_data])

# Reset the index of the final DataFrame
completed_data_30_to_90_days.reset_index(drop=True, inplace=True)
completed_data_30_to_90_days.head()

,user_programs_user_id,date,session_executions_updated_at,1 leg bridge (left)_reps_1,1 leg bridge (left)_reps_10,1 leg bridge (left)_reps_11,1 leg bridge (left)_reps_12,1 leg bridge (left)_reps_13,1 leg bridge (left)_reps_14,1 leg bridge (left)_reps_15,1 leg bridge (left)_reps_16,1 leg bridge (left)_reps_17,1 leg bridge (left)_reps_2,1 leg bridge (left)_reps_3,1 leg bridge (left)_reps_4,1 leg bridge (left)_reps_5,1 leg bridge (left)_reps_6,1 leg bridge (left)_reps_7,1 leg bridge (left)_reps_8,1 leg bridge (left)_reps_9,1 leg bridge (left)_time_1,1 leg bridge (left)_time_10,1 leg bridge (left)_time_11,1 leg bridge (left)_time_12,1 leg bridge (left)_time_13,1 leg bridge (left)_time_14,1 leg bridge (left)_time_15,1 leg bridge (left)_time_16,1 leg bridge (left)_time_17,1 leg bridge (left)_time_2,1 leg bridge (left)_time_3,1 leg bridge (left)_time_4,1 leg bridge (left)_time_5,1 leg bridge (left)_time_6,1 leg bridge (left)_time_7,1 leg bridge (left)_time_8,1 leg bridge (left)_time_9,1 leg bridge (right)_reps_1,1 leg bridge (right)_reps_10,1 leg bridge (right)_reps_11,1 leg bridge (right)_reps_12,1 leg bridge (right)_reps_13,1 leg bridge (right)_reps_14,1 leg bridge (right)_reps_15,1 leg bridge (right)_reps_16,1 leg bridge (right)_reps_17,1 leg bridge (right)_reps_2,1 leg bridge (right)_reps_3,1 leg bridge (right)_reps_4,1 leg bridge (right)_reps_5,1 leg bridge (right)_reps_6,1 leg bridge (right)_reps_7,1 leg bridge (right)_reps_8,1 leg bridge (right)_reps_9,1 leg bridge (right)_time_1,1 leg bridge (right)_time_10,1 leg bridge (right)_time_11,1 leg bridge (right)_time_12,1 leg bridge (right)_time_13,1 leg bridge (right)_time_14,1 leg bridge (right)_time_15,1 leg bridge (right)_time_16,1 leg bridge (right)_time_17,1 leg bridge (right)_time_2,1 leg bridge (right)_time_3,1 leg bridge (right)_time_4,1 leg bridge (right)_time_5,1 leg bridge (right)_time_6,1 leg bridge (right)_time_7,1 leg bridge (right)_time_8,1 leg bridge (right)_time_9,1 leg dead lift (left)_reps_1,1 leg dead lift (left)_reps_10,1 leg dead lift (left)_reps_11,1 leg dead lift (left)_reps_12,1 leg dead lift (left)_reps_13,1 leg dead lift (left)_reps_14,1 leg dead lift (left)_reps_15,1 leg dead lift (left)_reps_2,1 leg dead lift (left)_reps_3,1 leg dead lift (left)_reps_4,1 leg dead lift (left)_reps_5,1 leg dead lift (left)_reps_6,1 leg dead lift (left)_reps_7,1 leg dead lift (left)_reps_8,1 leg dead lift (left)_reps_9,1 leg dead lift (left)_time_1,1 leg dead lift (left)_time_10,1 leg dead lift (left)_time_11,1 leg dead lift (left)_time_12,1 leg dead lift (left)_time_13,1 leg dead lift (left)_time_14,1 leg dead lift (left)_time_15,1 leg dead lift (left)_time_2,1 leg dead lift (left)_time_3,1 leg dead lift (left)_time_4,1 leg dead lift (left)_time_5,1 leg dead lift (left)_time_6,1 leg dead lift (left)_time_7,1 leg dead lift (left)_time_8,1 leg dead lift (left)_time_9,1 leg dead lift (right)_reps_1,1 leg dead lift (right)_reps_10,1 leg dead lift (right)_reps_11,1 leg dead lift (right)_reps_12,1 leg dead lift (right)_reps_13,1 leg dead lift (right)_reps_14,1 leg dead lift (right)_reps_15,1 leg dead lift (right)_reps_2,1 leg dead lift (right)_reps_3,1 leg dead lift (right)_reps_4,1 leg dead lift (right)_reps_5,1 leg dead lift (right)_reps_6,1 leg dead lift (right)_reps_7,1 leg dead lift (right)_reps_8,1 leg dead lift (right)_reps_9,1 leg dead lift (right)_time_1,1 leg dead lift (right)_time_10,1 leg dead lift (right)_time_11,1 leg dead lift (right)_time_12,1 leg dead lift (right)_time_13,1 leg dead lift (right)_time_14,1 leg dead lift (right)_time_15,1 leg dead lift (right)_time_2,1 leg dead lift (right)_time_3,1 leg dead lift (right)_time_4,1 leg dead lift (right)_time_5,1 leg dead lift (right)_time_6,1 leg dead lift (right)_time_7,1 leg dead lift (right)_time_8,1 leg dead lift (right)_time_9,1 leg dead lift 1with KB (right)_reps_1,1 leg dead lift 1with KB (right)_reps_10,1 leg dead lift 1with KB (right)_reps_11,1 leg dead lift 1with KB (right)_reps_12,1 leg dead lift 1with K

In [16]:
completed_data_30_to_90_days.to_hdf('../data/completed_data_30_to_90_days_v1.h5', key='df')

In [9]:
completed_data_30_to_90_days = pd.read_hdf('../data/completed_data_30_to_90_days_v1.h5', key='df')
print(f'30-90 days dataset shape:', completed_data_30_to_90_days.shape)

30-90 days dataset shape: (49410, 8705)


In [8]:
# Ensure that 'date' is in datetime format
df_filtered_90_to_180_days['date'] = pd.to_datetime(df_filtered_90_to_180_days['date'])

# Get the first training day for each user
first_training_day_90_to_180 = df_filtered_90_to_180_days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day_90_to_180.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_90_to_180_days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day_90_to_180['user_programs_user_id'], first_training_day_90_to_180['first_training_date']):
    # Generate a date range from the first day up to 180 days later
    date_range = pd.date_range(start=first_day, periods=180)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_90_to_180_days[df_filtered_90_to_180_days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_90_to_180_days = pd.concat([completed_data_90_to_180_days, user_data])

# Reset the index of the final DataFrame
completed_data_90_to_180_days.reset_index(drop=True, inplace=True)
completed_data_90_to_180_days.head()

,user_programs_user_id,date,session_executions_updated_at,1 leg bridge (left)_reps_1,1 leg bridge (left)_reps_10,1 leg bridge (left)_reps_11,1 leg bridge (left)_reps_12,1 leg bridge (left)_reps_13,1 leg bridge (left)_reps_14,1 leg bridge (left)_reps_15,1 leg bridge (left)_reps_16,1 leg bridge (left)_reps_17,1 leg bridge (left)_reps_2,1 leg bridge (left)_reps_3,1 leg bridge (left)_reps_4,1 leg bridge (left)_reps_5,1 leg bridge (left)_reps_6,1 leg bridge (left)_reps_7,1 leg bridge (left)_reps_8,1 leg bridge (left)_reps_9,1 leg bridge (left)_time_1,1 leg bridge (left)_time_10,1 leg bridge (left)_time_11,1 leg bridge (left)_time_12,1 leg bridge (left)_time_13,1 leg bridge (left)_time_14,1 leg bridge (left)_time_15,1 leg bridge (left)_time_16,1 leg bridge (left)_time_17,1 leg bridge (left)_time_2,1 leg bridge (left)_time_3,1 leg bridge (left)_time_4,1 leg bridge (left)_time_5,1 leg bridge (left)_time_6,1 leg bridge (left)_time_7,1 leg bridge (left)_time_8,1 leg bridge (left)_time_9,1 leg bridge (right)_reps_1,1 leg bridge (right)_reps_10,1 leg bridge (right)_reps_11,1 leg bridge (right)_reps_12,1 leg bridge (right)_reps_13,1 leg bridge (right)_reps_14,1 leg bridge (right)_reps_15,1 leg bridge (right)_reps_16,1 leg bridge (right)_reps_17,1 leg bridge (right)_reps_2,1 leg bridge (right)_reps_3,1 leg bridge (right)_reps_4,1 leg bridge (right)_reps_5,1 leg bridge (right)_reps_6,1 leg bridge (right)_reps_7,1 leg bridge (right)_reps_8,1 leg bridge (right)_reps_9,1 leg bridge (right)_time_1,1 leg bridge (right)_time_10,1 leg bridge (right)_time_11,1 leg bridge (right)_time_12,1 leg bridge (right)_time_13,1 leg bridge (right)_time_14,1 leg bridge (right)_time_15,1 leg bridge (right)_time_16,1 leg bridge (right)_time_17,1 leg bridge (right)_time_2,1 leg bridge (right)_time_3,1 leg bridge (right)_time_4,1 leg bridge (right)_time_5,1 leg bridge (right)_time_6,1 leg bridge (right)_time_7,1 leg bridge (right)_time_8,1 leg bridge (right)_time_9,1 leg dead lift (left)_reps_1,1 leg dead lift (left)_reps_10,1 leg dead lift (left)_reps_11,1 leg dead lift (left)_reps_12,1 leg dead lift (left)_reps_13,1 leg dead lift (left)_reps_14,1 leg dead lift (left)_reps_15,1 leg dead lift (left)_reps_2,1 leg dead lift (left)_reps_3,1 leg dead lift (left)_reps_4,1 leg dead lift (left)_reps_5,1 leg dead lift (left)_reps_6,1 leg dead lift (left)_reps_7,1 leg dead lift (left)_reps_8,1 leg dead lift (left)_reps_9,1 leg dead lift (left)_time_1,1 leg dead lift (left)_time_10,1 leg dead lift (left)_time_11,1 leg dead lift (left)_time_12,1 leg dead lift (left)_time_13,1 leg dead lift (left)_time_14,1 leg dead lift (left)_time_15,1 leg dead lift (left)_time_2,1 leg dead lift (left)_time_3,1 leg dead lift (left)_time_4,1 leg dead lift (left)_time_5,1 leg dead lift (left)_time_6,1 leg dead lift (left)_time_7,1 leg dead lift (left)_time_8,1 leg dead lift (left)_time_9,1 leg dead lift (right)_reps_1,1 leg dead lift (right)_reps_10,1 leg dead lift (right)_reps_11,1 leg dead lift (right)_reps_12,1 leg dead lift (right)_reps_13,1 leg dead lift (right)_reps_14,1 leg dead lift (right)_reps_15,1 leg dead lift (right)_reps_2,1 leg dead lift (right)_reps_3,1 leg dead lift (right)_reps_4,1 leg dead lift (right)_reps_5,1 leg dead lift (right)_reps_6,1 leg dead lift (right)_reps_7,1 leg dead lift (right)_reps_8,1 leg dead lift (right)_reps_9,1 leg dead lift (right)_time_1,1 leg dead lift (right)_time_10,1 leg dead lift (right)_time_11,1 leg dead lift (right)_time_12,1 leg dead lift (right)_time_13,1 leg dead lift (right)_time_14,1 leg dead lift (right)_time_15,1 leg dead lift (right)_time_2,1 leg dead lift (right)_time_3,1 leg dead lift (right)_time_4,1 leg dead lift (right)_time_5,1 leg dead lift (right)_time_6,1 leg dead lift (right)_time_7,1 leg dead lift (right)_time_8,1 leg dead lift (right)_time_9,1 leg dead lift 1with KB (right)_reps_1,1 leg dead lift 1with KB (right)_reps_10,1 leg dead lift 1with KB (right)_reps_11,1 leg dead lift 1with KB (right)_reps_12,1 leg dead lift 1with K

In [ ]:
completed_data_90_to_180_days.to_hdf('../data/completed_data_90_to_180_days_v1.h5', key='df')

In [10]:
completed_data_90_to_180_days = pd.read_hdf('../data/completed_data_90_to_180_days_v1.h5', key='df')
print(f'90-180 days dataset shape:', completed_data_90_to_180_days.shape)

90-180 days dataset shape: (102960, 8705)


In [30]:
# Ensure that 'date' is in datetime format
df_filtered_180_to_365_days['date'] = pd.to_datetime(df_filtered_180_to_365_days['date'])

# Get the first training day for each user
first_training_day_180_to_365 = df_filtered_180_to_365_days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day_180_to_365.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_180_to_365_days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day_180_to_365['user_programs_user_id'], first_training_day_180_to_365['first_training_date']):
    # Generate a date range from the first day up to 365 days later
    date_range = pd.date_range(start=first_day, periods=365)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_180_to_365_days[df_filtered_180_to_365_days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_180_to_365_days = pd.concat([completed_data_180_to_365_days, user_data])

# Reset the index of the final DataFrame
completed_data_180_to_365_days.reset_index(drop=True, inplace=True)
print(completed_data_180_to_365_days)

        user_programs_user_id       date session_executions_updated_at  \
0                         108 2021-06-11    2021-06-11 18:00:35.640406   
1                         108 2021-06-12                             0   
2                         108 2021-06-13                             0   
3                         108 2021-06-14                             0   
4                         108 2021-06-15                             0   
...                       ...        ...                           ...   
149280                  11835 2022-11-22                             0   
149281                  11835 2022-11-23                             0   
149282                  11835 2022-11-24                             0   
149283                  11835 2022-11-25                             0   
149284                  11835 2022-11-26                             0   

        1 leg bridge (left)_reps_1  1 leg bridge (left)_reps_10  \
0                              0.0          

In [ ]:
# Memory error
completed_data_180_to_365_days.to_hdf('../data/completed_data_180_to_365_days_v1.h5', key='df')

In [42]:
# Save the DataFrames to a CSV file
completed_data_30days.to_csv('../data/completed_data_30_days.csv', index=False)
completed_data_30_to_90_days.to_csv('../data/completed_data_30_to_90_days.csv', index=False)
completed_data_90_to_180_days.to_csv('../data/completed_data_90_to_180_days.csv', index=False)
completed_data_180_to_365_days.to_csv('../data/completed_data_180_to_365_days.csv', index=False)

In [ ]:
# TODAVIA NO ESTA HECHO!!
# Falta el dataset de 180-365 dias!!

# Read the datasets
completed_data_30days = pd.read_hdf('../data/completed_data_30days_v1.h5', key='df')
completed_data_30_to_90_days = pd.read_hdf('../data/completed_data_30_to_90_days_v1.h5', key='df')
completed_data_90_to_180_days = pd.read_hdf('../data/completed_data_90_to_180_days_v1.h5', key='df')
completed_data_180_to_365_days = pd.read_hdf('../data/completed_data_180_to_365_days_v1.h5', key='df')

print(f'30 days dataset shape:', completed_data_30days.shape)
print(f'30-90 days dataset shape:', completed_data_30_to_90_days.shape)
print(f'90-180 days dataset shape:', completed_data_90_to_180_days.shape)
print(f'180-365 days dataset shape:', completed_data_180_to_365_days.shape)

## **Time Series to Supervised Learning**

The goal of this process is to convert time series data into a format suitable for supervised learning models. This involves restructuring the dataset so that past observations are used as input features to predict future observations. Here's a step-by-step breakdown of what I have done to convert the time series data into a supervised learning format:

##### **Step 1: Loading the Dataset**
- **Purpose**: The first step is to load the dataset from an HDF5 file. The dataset `completed_data_30days` represents the data of users who have trained for up to 30 days.
- **Shape**: The dataset has a shape of `(49740, 8705)`, indicating that it contains 49,740 rows (observations) and 8,705 columns (features).

##### **Step 2: Converting Series to Supervised Learning**
- **Purpose**: The `series_to_supervised` function converts the time series data into a supervised learning format. This involves shifting the time series data to create input-output pairs, where `n_in` is the number of lag observations to use as inputs, and `n_out` is the number of future observations to predict.

- **Operation 1: Preparing the DataFrame**
    - **What It Does**: This operation initializes the variables needed for the transformation. It converts the input data into a DataFrame (`df`) and initializes lists to store the columns and names for the final supervised dataset.
    - **Timing**: The time taken for this operation is printed for performance monitoring.

- **Operation 2: Creating Input Sequences**
    - **What It Does**: This operation creates the input sequences by shifting the DataFrame by `i` time steps for each of the `n_in` lag observations. These shifted values become the input features for the supervised learning model.
    - **Timing**: The time taken for this operation is recorded and printed.

- **Operation 3: Creating Output Sequences**
    - **What It Does**: This operation creates the output sequences by shifting the DataFrame forward by `i` time steps for each of the `n_out` future observations. These shifted values become the target variables that the model will predict.
    - **Timing**: The time taken for this operation is also recorded and printed.

- **Operation 4: Combining Inputs and Outputs**
    - **What It Does**: Here, the shifted input and output sequences are combined into a single DataFrame (`agg`). This DataFrame now has the format required for supervised learning, where each row represents an instance with `n_in` inputs and `n_out` outputs.
    - **Timing**: The time taken for this operation is recorded.

##### **Step 3: Applying the Function in Chunks**
- Given the size of the dataset, the `series_to_supervised` function is applied in chunks to avoid memory issues. The dataset is split into four parts, each containing approximately 12,500 rows.

- **Example of the Process for the First Chunk:**
    - **Splitting**: The first 12,500 rows are selected from the original dataset.
    - **Transformation**: The `series_to_supervised` function is applied to this chunk, with `n_in=7` and `n_out=3`.
    - **Saving**: The transformed data is saved as a HDF5 file for further use.
    - **Shape**: The shape of the transformed dataset is displayed to confirm the correct transformation.

- **Process for Subsequent Chunks:**

    The same steps are repeated for the remaining chunks:
    - `df_test_30days_part1` (from the beginning to row 12,500)
    - `df_test_30days_part2` (rows 12,500 to 25,000)
    - `df_test_30days_part2` (rows 25,000 to 37,500)
    - `df_test_30days_part2` (row 37,500 till the end)

    Each chunk is processed, transformed into a supervised learning format, and saved as a HDF5 file.

##### **Conclusion**

This approach ensures that the large dataset is efficiently converted into a supervised learning format without overwhelming the system's memory. By breaking the dataset into manageable chunks, the process can handle large-scale time series data and prepare it for machine learning models that require structured input-output pairs.

In [2]:
completed_data_30days = pd.read_hdf('../data/completed_data_30days_v1.h5', key='df')
print(f'30 days dataset shape:', completed_data_30days.shape)

30 days dataset shape: (49740, 8705)


In [4]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    # Operation 1
    start_time = time.time()
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    print("--- %s seconds ---" % (time.time() - start_time))

    # Operation 2
    # input sequence (t-n, ... t-1)
    start_time = time.time()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [(f't-{i} {col}') for col in df.columns]
    print("--- %s seconds ---" % (time.time() - start_time))


    # Operation 3
    start_time = time.time()
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [(f't {col}') for col in df.columns]
        else:
            names += [(f't+{i} {col}') for col in df.columns] 
    print("--- %s seconds ---" % (time.time() - start_time))

    #Operation 4
    # put it all together
    start_time = time.time()
    agg = concat(cols, axis=1)
    agg.columns = names
    print("--- %s seconds ---" % (time.time() - start_time))

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)

    del start_time, n_vars, df, cols, names, data, n_in, n_out, dropnan, i

    return agg 

In [5]:
df_test_30days_part1 = completed_data_30days[:12500]

In [6]:
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed_data_30days_part1 = series_to_supervised(df_test_30days_part1, 7, 3)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[17,18,19,20,21,22,23]], axis=1, inplace=True)
reframed_data_30days_part1.head()

--- 8.392333984375e-05 seconds ---
--- 2.146376132965088 seconds ---
--- 1.0525977611541748 seconds ---
--- 9.581216812133789 seconds ---


,t-7 user_programs_user_id,t-7 date,t-7 session_executions_updated_at,t-7 1 leg bridge (left)_reps_1,t-7 1 leg bridge (left)_reps_10,t-7 1 leg bridge (left)_reps_11,t-7 1 leg bridge (left)_reps_12,t-7 1 leg bridge (left)_reps_13,t-7 1 leg bridge (left)_reps_14,t-7 1 leg bridge (left)_reps_15,t-7 1 leg bridge (left)_reps_16,t-7 1 leg bridge (left)_reps_17,t-7 1 leg bridge (left)_reps_2,t-7 1 leg bridge (left)_reps_3,t-7 1 leg bridge (left)_reps_4,t-7 1 leg bridge (left)_reps_5,t-7 1 leg bridge (left)_reps_6,t-7 1 leg bridge (left)_reps_7,t-7 1 leg bridge (left)_reps_8,t-7 1 leg bridge (left)_reps_9,t-7 1 leg bridge (left)_time_1,t-7 1 leg bridge (left)_time_10,t-7 1 leg bridge (left)_time_11,t-7 1 leg bridge (left)_time_12,t-7 1 leg bridge (left)_time_13,t-7 1 leg bridge (left)_time_14,t-7 1 leg bridge (left)_time_15,t-7 1 leg bridge (left)_time_16,t-7 1 leg bridge (left)_time_17,t-7 1 leg bridge (left)_time_2,t-7 1 leg bridge (left)_time_3,t-7 1 leg bridge (left)_time_4,t-7 1 leg bridge (left)_time_5,t-7 1 leg bridge (left)_time_6,t-7 1 leg bridge (left)_time_7,t-7 1 leg bridge (left)_time_8,t-7 1 leg bridge (left)_time_9,t-7 1 leg bridge (right)_reps_1,t-7 1 leg bridge (right)_reps_10,t-7 1 leg bridge (right)_reps_11,t-7 1 leg bridge (right)_reps_12,t-7 1 leg bridge (right)_reps_13,t-7 1 leg bridge (right)_reps_14,t-7 1 leg bridge (right)_reps_15,t-7 1 leg bridge (right)_reps_16,t-7 1 leg bridge (right)_reps_17,t-7 1 leg bridge (right)_reps_2,t-7 1 leg bridge (right)_reps_3,t-7 1 leg bridge (right)_reps_4,t-7 1 leg bridge (right)_reps_5,t-7 1 leg bridge (right)_reps_6,t-7 1 leg bridge (right)_reps_7,t-7 1 leg bridge (right)_reps_8,t-7 1 leg bridge (right)_reps_9,t-7 1 leg bridge (right)_time_1,t-7 1 leg bridge (right)_time_10,t-7 1 leg bridge (right)_time_11,t-7 1 leg bridge (right)_time_12,t-7 1 leg bridge (right)_time_13,t-7 1 leg bridge (right)_time_14,t-7 1 leg bridge (right)_time_15,t-7 1 leg bridge (right)_time_16,t-7 1 leg bridge (right)_time_17,t-7 1 leg bridge (right)_time_2,t-7 1 leg bridge (right)_time_3,t-7 1 leg bridge (right)_time_4,t-7 1 leg bridge (right)_time_5,t-7 1 leg bridge (right)_time_6,t-7 1 leg bridge (right)_time_7,t-7 1 leg bridge (right)_time_8,t-7 1 leg bridge (right)_time_9,t-7 1 leg dead lift (left)_reps_1,t-7 1 leg dead lift (left)_reps_10,t-7 1 leg dead lift (left)_reps_11,t-7 1 leg dead lift (left)_reps_12,t-7 1 leg dead lift (left)_reps_13,t-7 1 leg dead lift (left)_reps_14,t-7 1 leg dead lift (left)_reps_15,t-7 1 leg dead lift (left)_reps_2,t-7 1 leg dead lift (left)_reps_3,t-7 1 leg dead lift (left)_reps_4,t-7 1 leg dead lift (left)_reps_5,t-7 1 leg dead lift (left)_reps_6,t-7 1 leg dead lift (left)_reps_7,t-7 1 leg dead lift (left)_reps_8,t-7 1 leg dead lift (left)_reps_9,t-7 1 leg dead lift (left)_time_1,t-7 1 leg dead lift (left)_time_10,t-7 1 leg dead lift (left)_time_11,t-7 1 leg dead lift (left)_time_12,t-7 1 leg dead lift (left)_time_13,t-7 1 leg dead lift (left)_time_14,t-7 1 leg dead lift (left)_time_15,t-7 1 leg dead lift (left)_time_2,t-7 1 leg dead lift (left)_time_3,t-7 1 leg dead lift (left)_time_4,t-7 1 leg dead lift (left)_time_5,t-7 1 leg dead lift (left)_time_6,t-7 1 leg dead lift (left)_time_7,t-7 1 leg dead lift (left)_time_8,t-7 1 leg dead lift (left)_time_9,t-7 1 leg dead lift (right)_reps_1,t-7 1 leg dead lift (right)_reps_10,t-7 1 leg dead lift (right)_reps_11,t-7 1 leg dead lift (right)_reps_12,t-7 1 leg dead lift (right)_reps_13,t-7 1 leg dead lift (right)_reps_14,t-7 1 leg dead lift (right)_reps_15,t-7 1 leg dead lift (right)_reps_2,t-7 1 leg dead lift (right)_reps_3,t-7 1 leg dead lift (right)_reps_4,t-7 1 leg dead lift (right)_reps_5,t-7 1 leg dead lift (right)_reps_6,t-7 1 leg dead lift (right)_reps_7,t-7 1 leg dead lift (right)_reps_8,t-7 1 leg dead lift (right)_reps_9,t-7 1 leg dead lift (right)_time_1,t-7 1 leg dead lift (right)_time_10,t-7 1 leg dead lift (right)_time_11,t-7 1 leg dead lift (right)_time_12,t-7 1 leg dead lift (right)_time_13,t-7 1 leg dead

In [7]:
reframed_data_30days_part1.to_hdf('../data/reframed_data_30days_part1.h5', key='df')

In [ ]:
reframed_data_30days_part1 = pd.read_hdf('../data/reframed_data_30days_part1.h5')
reframed_data_30days_part1.shape

In [8]:
df_test_30days_part2 = completed_data_30days[12500:25000]

In [9]:
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed_data_30days_part2 = series_to_supervised(df_test_30days_part2, 7, 3)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[17,18,19,20,21,22,23]], axis=1, inplace=True)
reframed_data_30days_part2.head()

--- 6.508827209472656e-05 seconds ---
--- 2.1238210201263428 seconds ---
--- 1.0483052730560303 seconds ---
--- 9.616330862045288 seconds ---


,t-7 user_programs_user_id,t-7 date,t-7 session_executions_updated_at,t-7 1 leg bridge (left)_reps_1,t-7 1 leg bridge (left)_reps_10,t-7 1 leg bridge (left)_reps_11,t-7 1 leg bridge (left)_reps_12,t-7 1 leg bridge (left)_reps_13,t-7 1 leg bridge (left)_reps_14,t-7 1 leg bridge (left)_reps_15,t-7 1 leg bridge (left)_reps_16,t-7 1 leg bridge (left)_reps_17,t-7 1 leg bridge (left)_reps_2,t-7 1 leg bridge (left)_reps_3,t-7 1 leg bridge (left)_reps_4,t-7 1 leg bridge (left)_reps_5,t-7 1 leg bridge (left)_reps_6,t-7 1 leg bridge (left)_reps_7,t-7 1 leg bridge (left)_reps_8,t-7 1 leg bridge (left)_reps_9,t-7 1 leg bridge (left)_time_1,t-7 1 leg bridge (left)_time_10,t-7 1 leg bridge (left)_time_11,t-7 1 leg bridge (left)_time_12,t-7 1 leg bridge (left)_time_13,t-7 1 leg bridge (left)_time_14,t-7 1 leg bridge (left)_time_15,t-7 1 leg bridge (left)_time_16,t-7 1 leg bridge (left)_time_17,t-7 1 leg bridge (left)_time_2,t-7 1 leg bridge (left)_time_3,t-7 1 leg bridge (left)_time_4,t-7 1 leg bridge (left)_time_5,t-7 1 leg bridge (left)_time_6,t-7 1 leg bridge (left)_time_7,t-7 1 leg bridge (left)_time_8,t-7 1 leg bridge (left)_time_9,t-7 1 leg bridge (right)_reps_1,t-7 1 leg bridge (right)_reps_10,t-7 1 leg bridge (right)_reps_11,t-7 1 leg bridge (right)_reps_12,t-7 1 leg bridge (right)_reps_13,t-7 1 leg bridge (right)_reps_14,t-7 1 leg bridge (right)_reps_15,t-7 1 leg bridge (right)_reps_16,t-7 1 leg bridge (right)_reps_17,t-7 1 leg bridge (right)_reps_2,t-7 1 leg bridge (right)_reps_3,t-7 1 leg bridge (right)_reps_4,t-7 1 leg bridge (right)_reps_5,t-7 1 leg bridge (right)_reps_6,t-7 1 leg bridge (right)_reps_7,t-7 1 leg bridge (right)_reps_8,t-7 1 leg bridge (right)_reps_9,t-7 1 leg bridge (right)_time_1,t-7 1 leg bridge (right)_time_10,t-7 1 leg bridge (right)_time_11,t-7 1 leg bridge (right)_time_12,t-7 1 leg bridge (right)_time_13,t-7 1 leg bridge (right)_time_14,t-7 1 leg bridge (right)_time_15,t-7 1 leg bridge (right)_time_16,t-7 1 leg bridge (right)_time_17,t-7 1 leg bridge (right)_time_2,t-7 1 leg bridge (right)_time_3,t-7 1 leg bridge (right)_time_4,t-7 1 leg bridge (right)_time_5,t-7 1 leg bridge (right)_time_6,t-7 1 leg bridge (right)_time_7,t-7 1 leg bridge (right)_time_8,t-7 1 leg bridge (right)_time_9,t-7 1 leg dead lift (left)_reps_1,t-7 1 leg dead lift (left)_reps_10,t-7 1 leg dead lift (left)_reps_11,t-7 1 leg dead lift (left)_reps_12,t-7 1 leg dead lift (left)_reps_13,t-7 1 leg dead lift (left)_reps_14,t-7 1 leg dead lift (left)_reps_15,t-7 1 leg dead lift (left)_reps_2,t-7 1 leg dead lift (left)_reps_3,t-7 1 leg dead lift (left)_reps_4,t-7 1 leg dead lift (left)_reps_5,t-7 1 leg dead lift (left)_reps_6,t-7 1 leg dead lift (left)_reps_7,t-7 1 leg dead lift (left)_reps_8,t-7 1 leg dead lift (left)_reps_9,t-7 1 leg dead lift (left)_time_1,t-7 1 leg dead lift (left)_time_10,t-7 1 leg dead lift (left)_time_11,t-7 1 leg dead lift (left)_time_12,t-7 1 leg dead lift (left)_time_13,t-7 1 leg dead lift (left)_time_14,t-7 1 leg dead lift (left)_time_15,t-7 1 leg dead lift (left)_time_2,t-7 1 leg dead lift (left)_time_3,t-7 1 leg dead lift (left)_time_4,t-7 1 leg dead lift (left)_time_5,t-7 1 leg dead lift (left)_time_6,t-7 1 leg dead lift (left)_time_7,t-7 1 leg dead lift (left)_time_8,t-7 1 leg dead lift (left)_time_9,t-7 1 leg dead lift (right)_reps_1,t-7 1 leg dead lift (right)_reps_10,t-7 1 leg dead lift (right)_reps_11,t-7 1 leg dead lift (right)_reps_12,t-7 1 leg dead lift (right)_reps_13,t-7 1 leg dead lift (right)_reps_14,t-7 1 leg dead lift (right)_reps_15,t-7 1 leg dead lift (right)_reps_2,t-7 1 leg dead lift (right)_reps_3,t-7 1 leg dead lift (right)_reps_4,t-7 1 leg dead lift (right)_reps_5,t-7 1 leg dead lift (right)_reps_6,t-7 1 leg dead lift (right)_reps_7,t-7 1 leg dead lift (right)_reps_8,t-7 1 leg dead lift (right)_reps_9,t-7 1 leg dead lift (right)_time_1,t-7 1 leg dead lift (right)_time_10,t-7 1 leg dead lift (right)_time_11,t-7 1 leg dead lift (right)_time_12,t-7 1 leg dead lift (right)_time_13,t-7 1 leg dead

In [10]:
reframed_data_30days_part2.to_hdf('../data/reframed_data_30days_part2.h5', key='df')

In [ ]:
reframed_data_30days_part2 = pd.read_hdf('../data/reframed_data_30days_part2.h5')
reframed_data_30days_part2.shape

In [11]:
df_test_30days_part3 = completed_data_30days[25000:37500]

In [12]:
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed_data_30days_part3 = series_to_supervised(df_test_30days_part3, 7, 3)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[17,18,19,20,21,22,23]], axis=1, inplace=True)
reframed_data_30days_part3.head()

--- 6.604194641113281e-05 seconds ---
--- 2.125030279159546 seconds ---
--- 1.0544977188110352 seconds ---
--- 9.643107891082764 seconds ---


,t-7 user_programs_user_id,t-7 date,t-7 session_executions_updated_at,t-7 1 leg bridge (left)_reps_1,t-7 1 leg bridge (left)_reps_10,t-7 1 leg bridge (left)_reps_11,t-7 1 leg bridge (left)_reps_12,t-7 1 leg bridge (left)_reps_13,t-7 1 leg bridge (left)_reps_14,t-7 1 leg bridge (left)_reps_15,t-7 1 leg bridge (left)_reps_16,t-7 1 leg bridge (left)_reps_17,t-7 1 leg bridge (left)_reps_2,t-7 1 leg bridge (left)_reps_3,t-7 1 leg bridge (left)_reps_4,t-7 1 leg bridge (left)_reps_5,t-7 1 leg bridge (left)_reps_6,t-7 1 leg bridge (left)_reps_7,t-7 1 leg bridge (left)_reps_8,t-7 1 leg bridge (left)_reps_9,t-7 1 leg bridge (left)_time_1,t-7 1 leg bridge (left)_time_10,t-7 1 leg bridge (left)_time_11,t-7 1 leg bridge (left)_time_12,t-7 1 leg bridge (left)_time_13,t-7 1 leg bridge (left)_time_14,t-7 1 leg bridge (left)_time_15,t-7 1 leg bridge (left)_time_16,t-7 1 leg bridge (left)_time_17,t-7 1 leg bridge (left)_time_2,t-7 1 leg bridge (left)_time_3,t-7 1 leg bridge (left)_time_4,t-7 1 leg bridge (left)_time_5,t-7 1 leg bridge (left)_time_6,t-7 1 leg bridge (left)_time_7,t-7 1 leg bridge (left)_time_8,t-7 1 leg bridge (left)_time_9,t-7 1 leg bridge (right)_reps_1,t-7 1 leg bridge (right)_reps_10,t-7 1 leg bridge (right)_reps_11,t-7 1 leg bridge (right)_reps_12,t-7 1 leg bridge (right)_reps_13,t-7 1 leg bridge (right)_reps_14,t-7 1 leg bridge (right)_reps_15,t-7 1 leg bridge (right)_reps_16,t-7 1 leg bridge (right)_reps_17,t-7 1 leg bridge (right)_reps_2,t-7 1 leg bridge (right)_reps_3,t-7 1 leg bridge (right)_reps_4,t-7 1 leg bridge (right)_reps_5,t-7 1 leg bridge (right)_reps_6,t-7 1 leg bridge (right)_reps_7,t-7 1 leg bridge (right)_reps_8,t-7 1 leg bridge (right)_reps_9,t-7 1 leg bridge (right)_time_1,t-7 1 leg bridge (right)_time_10,t-7 1 leg bridge (right)_time_11,t-7 1 leg bridge (right)_time_12,t-7 1 leg bridge (right)_time_13,t-7 1 leg bridge (right)_time_14,t-7 1 leg bridge (right)_time_15,t-7 1 leg bridge (right)_time_16,t-7 1 leg bridge (right)_time_17,t-7 1 leg bridge (right)_time_2,t-7 1 leg bridge (right)_time_3,t-7 1 leg bridge (right)_time_4,t-7 1 leg bridge (right)_time_5,t-7 1 leg bridge (right)_time_6,t-7 1 leg bridge (right)_time_7,t-7 1 leg bridge (right)_time_8,t-7 1 leg bridge (right)_time_9,t-7 1 leg dead lift (left)_reps_1,t-7 1 leg dead lift (left)_reps_10,t-7 1 leg dead lift (left)_reps_11,t-7 1 leg dead lift (left)_reps_12,t-7 1 leg dead lift (left)_reps_13,t-7 1 leg dead lift (left)_reps_14,t-7 1 leg dead lift (left)_reps_15,t-7 1 leg dead lift (left)_reps_2,t-7 1 leg dead lift (left)_reps_3,t-7 1 leg dead lift (left)_reps_4,t-7 1 leg dead lift (left)_reps_5,t-7 1 leg dead lift (left)_reps_6,t-7 1 leg dead lift (left)_reps_7,t-7 1 leg dead lift (left)_reps_8,t-7 1 leg dead lift (left)_reps_9,t-7 1 leg dead lift (left)_time_1,t-7 1 leg dead lift (left)_time_10,t-7 1 leg dead lift (left)_time_11,t-7 1 leg dead lift (left)_time_12,t-7 1 leg dead lift (left)_time_13,t-7 1 leg dead lift (left)_time_14,t-7 1 leg dead lift (left)_time_15,t-7 1 leg dead lift (left)_time_2,t-7 1 leg dead lift (left)_time_3,t-7 1 leg dead lift (left)_time_4,t-7 1 leg dead lift (left)_time_5,t-7 1 leg dead lift (left)_time_6,t-7 1 leg dead lift (left)_time_7,t-7 1 leg dead lift (left)_time_8,t-7 1 leg dead lift (left)_time_9,t-7 1 leg dead lift (right)_reps_1,t-7 1 leg dead lift (right)_reps_10,t-7 1 leg dead lift (right)_reps_11,t-7 1 leg dead lift (right)_reps_12,t-7 1 leg dead lift (right)_reps_13,t-7 1 leg dead lift (right)_reps_14,t-7 1 leg dead lift (right)_reps_15,t-7 1 leg dead lift (right)_reps_2,t-7 1 leg dead lift (right)_reps_3,t-7 1 leg dead lift (right)_reps_4,t-7 1 leg dead lift (right)_reps_5,t-7 1 leg dead lift (right)_reps_6,t-7 1 leg dead lift (right)_reps_7,t-7 1 leg dead lift (right)_reps_8,t-7 1 leg dead lift (right)_reps_9,t-7 1 leg dead lift (right)_time_1,t-7 1 leg dead lift (right)_time_10,t-7 1 leg dead lift (right)_time_11,t-7 1 leg dead lift (right)_time_12,t-7 1 leg dead lift (right)_time_13,t-7 1 leg dead

In [13]:
reframed_data_30days_part3.to_hdf('../data/reframed_data_30days_part3.h5', key='df')

In [ ]:
reframed_data_30days_part3 = pd.read_hdf('../data/reframed_data_30days_part3.h5')
reframed_data_30days_part3.shape

In [14]:
df_test_30days_part4 = completed_data_30days[37500:]

In [15]:
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed_data_30days_part4 = series_to_supervised(df_test_30days_part4, 7, 3)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[17,18,19,20,21,22,23]], axis=1, inplace=True)
reframed_data_30days_part4.head()

--- 6.580352783203125e-05 seconds ---
--- 2.098238945007324 seconds ---
--- 1.0354220867156982 seconds ---
--- 9.449811935424805 seconds ---


,t-7 user_programs_user_id,t-7 date,t-7 session_executions_updated_at,t-7 1 leg bridge (left)_reps_1,t-7 1 leg bridge (left)_reps_10,t-7 1 leg bridge (left)_reps_11,t-7 1 leg bridge (left)_reps_12,t-7 1 leg bridge (left)_reps_13,t-7 1 leg bridge (left)_reps_14,t-7 1 leg bridge (left)_reps_15,t-7 1 leg bridge (left)_reps_16,t-7 1 leg bridge (left)_reps_17,t-7 1 leg bridge (left)_reps_2,t-7 1 leg bridge (left)_reps_3,t-7 1 leg bridge (left)_reps_4,t-7 1 leg bridge (left)_reps_5,t-7 1 leg bridge (left)_reps_6,t-7 1 leg bridge (left)_reps_7,t-7 1 leg bridge (left)_reps_8,t-7 1 leg bridge (left)_reps_9,t-7 1 leg bridge (left)_time_1,t-7 1 leg bridge (left)_time_10,t-7 1 leg bridge (left)_time_11,t-7 1 leg bridge (left)_time_12,t-7 1 leg bridge (left)_time_13,t-7 1 leg bridge (left)_time_14,t-7 1 leg bridge (left)_time_15,t-7 1 leg bridge (left)_time_16,t-7 1 leg bridge (left)_time_17,t-7 1 leg bridge (left)_time_2,t-7 1 leg bridge (left)_time_3,t-7 1 leg bridge (left)_time_4,t-7 1 leg bridge (left)_time_5,t-7 1 leg bridge (left)_time_6,t-7 1 leg bridge (left)_time_7,t-7 1 leg bridge (left)_time_8,t-7 1 leg bridge (left)_time_9,t-7 1 leg bridge (right)_reps_1,t-7 1 leg bridge (right)_reps_10,t-7 1 leg bridge (right)_reps_11,t-7 1 leg bridge (right)_reps_12,t-7 1 leg bridge (right)_reps_13,t-7 1 leg bridge (right)_reps_14,t-7 1 leg bridge (right)_reps_15,t-7 1 leg bridge (right)_reps_16,t-7 1 leg bridge (right)_reps_17,t-7 1 leg bridge (right)_reps_2,t-7 1 leg bridge (right)_reps_3,t-7 1 leg bridge (right)_reps_4,t-7 1 leg bridge (right)_reps_5,t-7 1 leg bridge (right)_reps_6,t-7 1 leg bridge (right)_reps_7,t-7 1 leg bridge (right)_reps_8,t-7 1 leg bridge (right)_reps_9,t-7 1 leg bridge (right)_time_1,t-7 1 leg bridge (right)_time_10,t-7 1 leg bridge (right)_time_11,t-7 1 leg bridge (right)_time_12,t-7 1 leg bridge (right)_time_13,t-7 1 leg bridge (right)_time_14,t-7 1 leg bridge (right)_time_15,t-7 1 leg bridge (right)_time_16,t-7 1 leg bridge (right)_time_17,t-7 1 leg bridge (right)_time_2,t-7 1 leg bridge (right)_time_3,t-7 1 leg bridge (right)_time_4,t-7 1 leg bridge (right)_time_5,t-7 1 leg bridge (right)_time_6,t-7 1 leg bridge (right)_time_7,t-7 1 leg bridge (right)_time_8,t-7 1 leg bridge (right)_time_9,t-7 1 leg dead lift (left)_reps_1,t-7 1 leg dead lift (left)_reps_10,t-7 1 leg dead lift (left)_reps_11,t-7 1 leg dead lift (left)_reps_12,t-7 1 leg dead lift (left)_reps_13,t-7 1 leg dead lift (left)_reps_14,t-7 1 leg dead lift (left)_reps_15,t-7 1 leg dead lift (left)_reps_2,t-7 1 leg dead lift (left)_reps_3,t-7 1 leg dead lift (left)_reps_4,t-7 1 leg dead lift (left)_reps_5,t-7 1 leg dead lift (left)_reps_6,t-7 1 leg dead lift (left)_reps_7,t-7 1 leg dead lift (left)_reps_8,t-7 1 leg dead lift (left)_reps_9,t-7 1 leg dead lift (left)_time_1,t-7 1 leg dead lift (left)_time_10,t-7 1 leg dead lift (left)_time_11,t-7 1 leg dead lift (left)_time_12,t-7 1 leg dead lift (left)_time_13,t-7 1 leg dead lift (left)_time_14,t-7 1 leg dead lift (left)_time_15,t-7 1 leg dead lift (left)_time_2,t-7 1 leg dead lift (left)_time_3,t-7 1 leg dead lift (left)_time_4,t-7 1 leg dead lift (left)_time_5,t-7 1 leg dead lift (left)_time_6,t-7 1 leg dead lift (left)_time_7,t-7 1 leg dead lift (left)_time_8,t-7 1 leg dead lift (left)_time_9,t-7 1 leg dead lift (right)_reps_1,t-7 1 leg dead lift (right)_reps_10,t-7 1 leg dead lift (right)_reps_11,t-7 1 leg dead lift (right)_reps_12,t-7 1 leg dead lift (right)_reps_13,t-7 1 leg dead lift (right)_reps_14,t-7 1 leg dead lift (right)_reps_15,t-7 1 leg dead lift (right)_reps_2,t-7 1 leg dead lift (right)_reps_3,t-7 1 leg dead lift (right)_reps_4,t-7 1 leg dead lift (right)_reps_5,t-7 1 leg dead lift (right)_reps_6,t-7 1 leg dead lift (right)_reps_7,t-7 1 leg dead lift (right)_reps_8,t-7 1 leg dead lift (right)_reps_9,t-7 1 leg dead lift (right)_time_1,t-7 1 leg dead lift (right)_time_10,t-7 1 leg dead lift (right)_time_11,t-7 1 leg dead lift (right)_time_12,t-7 1 leg dead lift (right)_time_13,t-7 1 leg dead

In [16]:
reframed_data_30days_part4.to_hdf('../data/reframed_data_30days_part4.h5', key='df')

In [ ]:
reframed_data_30days_part4 = pd.read_hdf('../data/reframed_data_30days_part4.h5')
reframed_data_30days_part4.shape

### **Merging the Datasets**

The purpose of this process was to consolidate multiple parts of a large dataset into a single, unified dataset. Initially, the time series data from `completed_data_30days` was split into four separate parts, each processed individually to convert it into a supervised learning format. These parts were then read back into memory from HDF5 files.

The key step was the **concatenation** of these four datasets. By merging them into a single DataFrame, I created a comprehensive dataset that includes all rows and features across the original splits. This step was crucial for ensuring that the entire dataset could be utilized together for further analysis or modeling, preserving the continuity and integrity of the data.

Finally, the merged dataset was saved as a single HDF5 file, `merged_data_30days.h5`, allowing for easy access and use in subsequent stages of the project.

In [3]:
reframed_data_30days_part1 = pd.read_hdf('../data/reframed_data_30days_part1.h5')
reframed_data_30days_part2 = pd.read_hdf('../data/reframed_data_30days_part2.h5')
reframed_data_30days_part3 = pd.read_hdf('../data/reframed_data_30days_part3.h5')
reframed_data_30days_part4 = pd.read_hdf('../data/reframed_data_30days_part4.h5')

In [3]:
# Concat the DataFrames
merged_reframed_data_30days = pd.concat([reframed_data_30days_part1, reframed_data_30days_part2,
                                        reframed_data_30days_part3, reframed_data_30days_part4], 
                                        axis=0, ignore_index=True)
merged_reframed_data_30days.shape

(49704, 87050)

In [4]:
del reframed_data_30days_part1, reframed_data_30days_part2, reframed_data_30days_part3, reframed_data_30days_part4

In [8]:
# Identify columns with mixed types
for col in merged_reframed_data_30days.columns:
    if merged_reframed_data_30days[col].map(type).nunique() > 1:
        print(f"Converting column '{col}' to string due to mixed types.")
        merged_reframed_data_30days[col] = merged_reframed_data_30days[col].astype(str)

merged_reframed_data_30days.to_parquet('../data/merged_reframed_data_30days.parquet', engine='pyarrow')

Converting column 't-7 session_executions_updated_at' to string due to mixed types.
Converting column 't-7 month' to string due to mixed types.
Converting column 't-7 body_parts_focused' to string due to mixed types.
Converting column 't-7 exercises_met_multiplier' to string due to mixed types.
Converting column 't-7 exercises_muscles' to string due to mixed types.
Converting column 't-7 session_executions_discarded' to string due to mixed types.
Converting column 't-7 session_executions_imported' to string due to mixed types.
Converting column 't-7 user_programs_active' to string due to mixed types.
Converting column 't-7 user_programs_completed' to string due to mixed types.
Converting column 't-7 users_gender' to string due to mixed types.
Converting column 't-7 users_newsletter_subscription' to string due to mixed types.
Converting column 't-7 users_notifications_setting' to string due to mixed types.
Converting column 't-7 users_country' to string due to mixed types.
Converting co

In [6]:
merged_reframed_data_30days = pd.read_parquet('../data/merged_reframed_data_30days.parquet')

merged_reframed_data_30days.shape

(49704, 87050)

In [8]:
merged_reframed_data_30days

,t-7 user_programs_user_id,t-7 date,t-7 session_executions_updated_at,t-7 1 leg bridge (left)_reps_1,t-7 1 leg bridge (left)_reps_10,t-7 1 leg bridge (left)_reps_11,t-7 1 leg bridge (left)_reps_12,t-7 1 leg bridge (left)_reps_13,t-7 1 leg bridge (left)_reps_14,t-7 1 leg bridge (left)_reps_15,t-7 1 leg bridge (left)_reps_16,t-7 1 leg bridge (left)_reps_17,t-7 1 leg bridge (left)_reps_2,t-7 1 leg bridge (left)_reps_3,t-7 1 leg bridge (left)_reps_4,t-7 1 leg bridge (left)_reps_5,t-7 1 leg bridge (left)_reps_6,t-7 1 leg bridge (left)_reps_7,t-7 1 leg bridge (left)_reps_8,t-7 1 leg bridge (left)_reps_9,t-7 1 leg bridge (left)_time_1,t-7 1 leg bridge (left)_time_10,t-7 1 leg bridge (left)_time_11,t-7 1 leg bridge (left)_time_12,t-7 1 leg bridge (left)_time_13,t-7 1 leg bridge (left)_time_14,t-7 1 leg bridge (left)_time_15,t-7 1 leg bridge (left)_time_16,t-7 1 leg bridge (left)_time_17,t-7 1 leg bridge (left)_time_2,t-7 1 leg bridge (left)_time_3,t-7 1 leg bridge (left)_time_4,t-7 1 leg bridge (left)_time_5,t-7 1 leg bridge (left)_time_6,t-7 1 leg bridge (left)_time_7,t-7 1 leg bridge (left)_time_8,t-7 1 leg bridge (left)_time_9,t-7 1 leg bridge (right)_reps_1,t-7 1 leg bridge (right)_reps_10,t-7 1 leg bridge (right)_reps_11,t-7 1 leg bridge (right)_reps_12,t-7 1 leg bridge (right)_reps_13,t-7 1 leg bridge (right)_reps_14,t-7 1 leg bridge (right)_reps_15,t-7 1 leg bridge (right)_reps_16,t-7 1 leg bridge (right)_reps_17,t-7 1 leg bridge (right)_reps_2,t-7 1 leg bridge (right)_reps_3,t-7 1 leg bridge (right)_reps_4,t-7 1 leg bridge (right)_reps_5,t-7 1 leg bridge (right)_reps_6,t-7 1 leg bridge (right)_reps_7,t-7 1 leg bridge (right)_reps_8,t-7 1 leg bridge (right)_reps_9,t-7 1 leg bridge (right)_time_1,t-7 1 leg bridge (right)_time_10,t-7 1 leg bridge (right)_time_11,t-7 1 leg bridge (right)_time_12,t-7 1 leg bridge (right)_time_13,t-7 1 leg bridge (right)_time_14,t-7 1 leg bridge (right)_time_15,t-7 1 leg bridge (right)_time_16,t-7 1 leg bridge (right)_time_17,t-7 1 leg bridge (right)_time_2,t-7 1 leg bridge (right)_time_3,t-7 1 leg bridge (right)_time_4,t-7 1 leg bridge (right)_time_5,t-7 1 leg bridge (right)_time_6,t-7 1 leg bridge (right)_time_7,t-7 1 leg bridge (right)_time_8,t-7 1 leg bridge (right)_time_9,t-7 1 leg dead lift (left)_reps_1,t-7 1 leg dead lift (left)_reps_10,t-7 1 leg dead lift (left)_reps_11,t-7 1 leg dead lift (left)_reps_12,t-7 1 leg dead lift (left)_reps_13,t-7 1 leg dead lift (left)_reps_14,t-7 1 leg dead lift (left)_reps_15,t-7 1 leg dead lift (left)_reps_2,t-7 1 leg dead lift (left)_reps_3,t-7 1 leg dead lift (left)_reps_4,t-7 1 leg dead lift (left)_reps_5,t-7 1 leg dead lift (left)_reps_6,t-7 1 leg dead lift (left)_reps_7,t-7 1 leg dead lift (left)_reps_8,t-7 1 leg dead lift (left)_reps_9,t-7 1 leg dead lift (left)_time_1,t-7 1 leg dead lift (left)_time_10,t-7 1 leg dead lift (left)_time_11,t-7 1 leg dead lift (left)_time_12,t-7 1 leg dead lift (left)_time_13,t-7 1 leg dead lift (left)_time_14,t-7 1 leg dead lift (left)_time_15,t-7 1 leg dead lift (left)_time_2,t-7 1 leg dead lift (left)_time_3,t-7 1 leg dead lift (left)_time_4,t-7 1 leg dead lift (left)_time_5,t-7 1 leg dead lift (left)_time_6,t-7 1 leg dead lift (left)_time_7,t-7 1 leg dead lift (left)_time_8,t-7 1 leg dead lift (left)_time_9,t-7 1 leg dead lift (right)_reps_1,t-7 1 leg dead lift (right)_reps_10,t-7 1 leg dead lift (right)_reps_11,t-7 1 leg dead lift (right)_reps_12,t-7 1 leg dead lift (right)_reps_13,t-7 1 leg dead lift (right)_reps_14,t-7 1 leg dead lift (right)_reps_15,t-7 1 leg dead lift (right)_reps_2,t-7 1 leg dead lift (right)_reps_3,t-7 1 leg dead lift (right)_reps_4,t-7 1 leg dead lift (right)_reps_5,t-7 1 leg dead lift (right)_reps_6,t-7 1 leg dead lift (right)_reps_7,t-7 1 leg dead lift (right)_reps_8,t-7 1 leg dead lift (right)_reps_9,t-7 1 leg dead lift (right)_time_1,t-7 1 leg dead lift (right)_time_10,t-7 1 leg dead lift (right)_time_11,t-7 1 leg dead lift (right)_time_12,t-7 1 leg dead lift (right)_time_13,t-7 1 leg dead